In [4]:
import requests
import bs4
import re
import pandas as pd
import numpy as np
import time
import chardet
import os
from urllib.error import HTTPError
from requests.exceptions import HTTPError

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent

from requests.adapters import HTTPAdapter, Retry

In [29]:
%%time

albumTitleList, artistNameList, albumYearList, albumScoreList, albumGenreList = [], [], [], [], []
albumBNMList, albumBNRList, reviewAuthorList, artistLabelList, reviewDateTimeList, reviewAbstractList = [], [], [], [], [], []
reviewBodyList, reviewWordCountList, reviewCharCountList, urlList = [], [], [], []

#2107 problem
pageNumber = 1
baseUrl = 'https://pitchfork.com/'

useragent = UserAgent()
profile = webdriver.FirefoxProfile()

options = webdriver.FirefoxOptions()
options.add_argument("--headless")

profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile, options=options)
#driver.maximize_window()

retries = Retry(total=10,
                backoff_factor=1,
                status_forcelist=[ 500, 502, 503, 504 ])

s = requests.Session()
s.mount('http://', HTTPAdapter(max_retries=retries))
s.mount('https://', HTTPAdapter(max_retries=retries))


headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.110 Safari/537.36"}

i = 0

try:
    while True:

        driver.get(f'{baseUrl}/reviews/albums/?page={pageNumber}')

        albumsSoup = bs4.BeautifulSoup(driver.page_source, 'lxml')
        
        time.sleep(2)
        
        if len(albumsSoup.select("a[class='review__link']")) < 1:
            print(pageNumber)
            pageNumber = pageNumber - 1
            i = i + 1
            if i == 5:
                break

        for album in albumsSoup.select("a[class='review__link']"):
            
            reviewUrl = f"{baseUrl}{album.attrs['href']}"   
            resReview = s.get(reviewUrl)
            resReview.raise_for_status()
            reviewSoup = bs4.BeautifulSoup(resReview.text, 'lxml')
            
            if reviewUrl in urlList:
                continue
            
            for i in range(str(reviewSoup.select("h1[class='single-album-tombstone__review-title']")).count('single-album-tombstone__review-title')):
                # Album Title
                try:
                    albumTitleList.append(reviewSoup.select("h1[class='single-album-tombstone__review-title']")[i].text)
                except:
                    albumTitleList.append(np.nan)

                # Artist Name
                try:
                    artistNameList.append(reviewSoup.select("ul[class='artist-links artist-list single-album-tombstone__artist-links']")[i].text)
                except IndexError:
                    artistNameList.append(np.nan)

                # Year Album Released
                try:
                    albumYearList.append(reviewSoup.select("span[class='single-album-tombstone__meta-year']")[i].text.split('•')[1].strip())
                except IndexError:
                    albumYearList.append(np.nan)

                # Album Score
                try:
                    albumScoreList.append(reviewSoup.select("span[class='score']")[i].text)
                except:
                    albumScoreList.append(np.nan)

                # Best new music
                if 'Best new music' in str(reviewSoup.select("div[class='single-album-tombstone']")[i].text):
                    albumBNMList.append(1)
                else:
                    albumBNMList.append(0)
                    
                # Best new reissue
                if 'Best new reissue' in str(reviewSoup.select("div[class='single-album-tombstone']")[i].text):
                    albumBNRList.append(1)
                else:
                    albumBNRList.append(0)

                # Genre
                try:
                    stringTemp = ''
                    for i in range(str(reviewSoup.select("a[class='genre-list__link']")).count('href')):
                        stringTemp = stringTemp + reviewSoup.select("a[class='genre-list__link']")[i].text + ' / '
                    albumGenreList.append(stringTemp.rsplit(' / ', 1)[0])
                except IndexError:
                    albumGenreList.append(np.nan)

                # Artist's Label
                try:
                    artistLabelList.append(reviewSoup.select("li[class='labels-list__item']")[i].text)
                except IndexError:
                    artistLabelList.append(np.nan)

                # Author of the review
                try:
                    reviewAuthorList.append(reviewSoup.select("a[class='authors-detail__display-name']")[0].text)
                except IndexError:
                    reviewAuthorList.append(np.nan)

                # Review Publication Time
                try:
                    reviewDateTimeList.append(reviewSoup.select("time[class='pub-date']")[0].attrs['datetime'])
                except IndexError:
                    reviewDateTimeList.append(np.nan)

                # Review Abstract
                try:
                    reviewAbstractList.append(reviewSoup.select("div[class='review-detail__abstract'] p")[0].text)
                except IndexError:
                    reviewAbstractList.append(np.nan)

                # Review Contents
                reviewBody = ''
                for p in reviewSoup.select("div[class='contents dropcap'] p"):
                    reviewBody = reviewBody + p.text.replace('\xa0', ' ').strip()

                if not reviewBody:
                    for p in reviewSoup.select("div[class='contents'] p"):
                        reviewBody = reviewBody + p.text.replace('\xa0', ' ').strip()

                reviewBodyList.append(reviewBody)
                reviewWordCountList.append(len(reviewBody.split()))
                reviewCharCountList.append(len(reviewBody))

                urlList.append(reviewUrl)
                
                i = 0

        if (pageNumber % 100 == 0):
            print(pageNumber)
            
        pageNumber = pageNumber + 1


except Exception as e:
    driver.quit() 
    if e.code == 404:
        print('No more pages :)')
    else:
        print(e.code)

driver.quit()    
    
df = pd.DataFrame({'title': albumTitleList,
                   'artist': artistNameList,
                   'year': albumYearList,
                   'score': albumScoreList,
                   'best new music': albumBNMList,
                   'best new reissue': albumBNRList,
                   'genre': albumGenreList,
                   'label': artistLabelList,
                   'author': reviewAuthorList,
                   'abstract': reviewAbstractList,
                   'review': reviewBodyList,
                   'word count': reviewWordCountList,
                   'char count': reviewCharCountList,
                   'publication date': reviewDateTimeList,
                   'url' : urlList})

# Path to save to
save_to_path = 'D:\\CS\\Projects\\p4k\\reviewListUnClean'

df.to_csv(save_to_path, index=False)

100
200
300
400
500
600
700
800
900
989
1000
1100
1200
1300
1395
1400
1500
1600
1700
1800
1900
2000
2100
2117
2117
2117
2117
2117
CPU times: total: 22min 44s
Wall time: 5h 58min 33s
